In [2]:
from tensorflow.keras.layers import Dense, Flatten, Input 
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.xception import Xception, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
imageSize = [224, 224]

trainPath = r"/content/drive/MyDrive/Public datasets/Plant seedling/train"

testPath = r"/content/drive/MyDrive/Public datasets/Plant seedling/test"


In [5]:
xception = Xception(input_shape=imageSize + [3], weights='imagenet',include_top=False)

83697664/83683744 [==============================] - 1s 0us/step


In [6]:
# don't train existing weights
for layer in xception.layers:
  layer.trainable = False

In [7]:
# our layers - you can add more if you want
x = Flatten()(xception.output)

In [8]:
prediction = Dense(12, activation='softmax')(x)

In [9]:
# create a model object
model = Model(inputs=xception.input, outputs=prediction)

In [10]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

In [11]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [13]:
training_set = train_datagen.flow_from_directory(trainPath,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(testPath,
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 3805 images belonging to 12 classes.
Found 957 images belonging to 12 classes.


In [14]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=100,
  steps_per_epoch=len(training_set)//5,
  validation_steps=len(test_set)//5
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/100
23/23 [==============================] - 327s 14s/step - loss: 4.5459 - accuracy: 0.3383 - val_loss: 2.7742 - val_accuracy: 0.5365
Epoch 2/100
23/23 [==============================] - 257s 11s/step - loss: 2.7006 - accuracy: 0.5512 - val_loss: 2.6233 - val_accuracy: 0.5729
Epoch 3/100
23/23 [==============================] - 195s 8s/step - loss: 1.8814 - accuracy: 0.6399 - val_loss: 2.1104 - val_accuracy: 0.6562
Epoch 4/100
23/23 [==============================] - 166s 7s/step - loss: 2.0237 - accuracy: 0.6454 - val_loss: 2.0049 - val_accuracy: 0.7240
Epoch 5/100
23/23 [==============================] - 136s 6s/step - loss: 1.9850 - accuracy: 0.6780 - val_loss: 1.7057 - val_accuracy: 0.6771
Epoch 6/100
23/23 [==============================] - 114s 5s/step - loss: 1.7512 - accuracy: 0.7188 - val_loss: 2.8943 - val_accuracy: 0.6458
Epoch 7/100
23/23 [==============================] - 101s 4s/step - loss: 2.1030 - accuracy: 0.7011 - val_loss: 2.2158 - val_accuracy: 0.6667
Epoc

In [15]:
model.save('seedling_exception.h5')

# Testing

In [7]:
#import load_model class for loading h5 file
from tensorflow.keras.models import load_model
#import image class to process the images
from tensorflow.keras.preprocessing import image
#from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.xception import Xception, preprocess_input
import numpy as np

In [8]:
#load saved model file
model=load_model('seedling_exception.h5')

In [9]:
#load one random image from local system
img=image.load_img(r"D:\data science\Plant seedling classification\Data\test\Charlock\0c4a435c7.png",target_size=(224,224))

In [10]:
#convert image to array format
x=image.img_to_array(img)

In [11]:
x.shape

(224, 224, 3)

In [12]:
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [13]:
output=np.argmax(model.predict(img_data), axis=1)
output

array([1], dtype=int64)